# 1. 사용자 입력 받기
IATA 코드로 여행 구간을 입력받습니다.  

In [10]:
def get_user_input():
    """
    사용자로부터 입력값을 받습니다.
    """
    print("사용자 정보를 입력하세요:")
    seat_type = input("탑승 클래스 (Economy Class, Premium Economy, Business Class, First Class): ").strip()
    route = input("여행 구간 (예: DXB to LHR): ").strip()
    analyze_reviews = input("리뷰 분석 여부 (Yes/No): ").strip().lower() == "yes"
    
    print("서비스 선호도를 입력하세요 (1~5 순위):")
    preferences = {
        "Seat Comfort": int(input("좌석 편안함 (1~5): ").strip()),
        "Cabin Staff Service": int(input("객실 승무원 서비스 (1~5): ").strip()),
        "Food & Beverages": int(input("기내 음식 및 음료 (1~5): ").strip()),
        "Inflight Entertainment": int(input("기내 엔터테인먼트 (1~5): ").strip()),
        "Ground Service": int(input("지상 직원 서비스 (1~5): ").strip()),
        "Wifi & Connectivity": int(input("기내 와이파이 (1~5): ").strip()),
        "Value For Money": int(input("가격 대비 가치 (1~5): ").strip()),
    }
    
    return {
        "seat_type": seat_type,
        "route": route,
        "analyze_reviews": analyze_reviews,
        "preferences": preferences,
    }

user_input = get_user_input()


사용자 정보를 입력하세요:
서비스 선호도를 입력하세요 (1~5 순위):


# 2. OpenFlights 데이터셋 로드
IATA 코드와 도시명을 매핑하기 위한 OpenFlights 데이터셋을 로드합니다.


In [11]:
import pandas as pd

def load_airport_mapping(file_path):
    """
    OpenFlights 데이터셋에서 IATA 코드와 도시 이름 매핑 생성.
    """
    columns = ["Airport ID", "Name", "City", "Country", "IATA", "ICAO", "Latitude",
               "Longitude", "Altitude", "Timezone", "DST", "Tz database time zone", "Type", "Source"]
    airport_data = pd.read_csv(file_path, header=None, names=columns)
    return airport_data[["IATA", "City"]].dropna().set_index("IATA").to_dict()["City"]

file_path_airports = "/home/gyuha_lee/Opensource_dev/get_information/mine/airports.dat"
airport_city_mapping = load_airport_mapping(file_path_airports)


# 3. 데이터 로드 및 전처리
리뷰 데이터셋을 로드하고 필요한 열만 유지합니다.


In [12]:
import json

def preprocess_data(file_path):
    """
    데이터셋 로드 및 전처리.
    """
    with open(file_path, 'r') as file:
        data = json.load(file)
    df = pd.DataFrame(data)

    # 필요한 열만 유지
    required_columns = [
        "Airline Name", "Route", "Seat Type", "Seat Comfort",
        "Cabin Staff Service", "Food & Beverages", "Inflight Entertainment",
        "Ground Service", "Wifi & Connectivity", "Value For Money", "Review Content"
    ]
    df = df[required_columns]
    return df

file_path_reviews = "/home/gyuha_lee/Opensource_dev/get_information/mine/all_airline_reviews.json"
df = preprocess_data(file_path_reviews)


# 4. Route 정규화 및 필터링
IATA 코드 기반 및 도시명 기반으로 데이터를 필터링합니다.  
결과를 병합하고 중복 데이터를 제거합니다.


In [13]:
def normalize_route(route, mapping, to_city=False):
    """
    Route 값을 IATA 코드 또는 도시 이름으로 정규화.
    """
    if pd.isnull(route) or isinstance(route, (int, float)):
        return ""
    if "via" in route.lower():
        route = route.split("via")[0].strip()
    route_parts = route.lower().replace("to", "").split()
    
    # IATA 코드 <-> 도시명 변환
    normalized_parts = []
    for part in route_parts:
        if to_city:  # IATA 코드를 도시명으로 변환
            normalized_parts.append(mapping.get(part.upper(), part))
        else:  # 그대로 IATA 코드 유지
            normalized_parts.append(part.upper())

    # 두 개의 구간만 있는 경우 " to "로 연결
    if len(normalized_parts) == 2:
        return " to ".join(normalized_parts).strip()
    return ""

def match_route(input_route, data_route):
    """
    입력 Route와 데이터 Route를 양방향으로 비교.
    """
    try:
        input_start, input_end = input_route.split(" to ")
        data_start, data_end = data_route.split(" to ")
        return (
            (input_start == data_start and input_end == data_end) or
            (input_start == data_end and input_end == data_start)
        )
    except ValueError:
        return False

def filter_data_by_iata(df, user_input):
    """
    IATA 코드 기반 필터링.
    """
    seat_type = user_input["seat_type"].strip().lower()
    input_route = user_input["route"].strip()

    # 입력 Route 정규화 (IATA 코드 기준)
    normalized_input_route = normalize_route(input_route, airport_city_mapping, to_city=False)

    # 데이터셋의 Route 정규화 (IATA 코드 기준)
    df["Normalized Route (IATA)"] = df["Route"].apply(lambda r: normalize_route(r, airport_city_mapping, to_city=False))

    # 필터링
    filtered_df = df[df["Seat Type"].str.strip().str.lower() == seat_type]
    filtered_df = filtered_df[filtered_df["Normalized Route (IATA)"].apply(lambda r: match_route(normalized_input_route, r))]
    return filtered_df

def filter_data_by_city(df, user_input, mapping):
    """
    도시명 기반 필터링.
    """
    seat_type = user_input["seat_type"].strip().lower()
    input_route = user_input["route"].strip()

    # 입력 Route 정규화 (도시명 기준)
    normalized_input_route = normalize_route(input_route, mapping, to_city=True)

    # 데이터셋의 Route 정규화 (도시명 기준)
    df["Normalized Route (City)"] = df["Route"].apply(lambda r: normalize_route(r, mapping, to_city=True))

    # 필터링
    filtered_df = df[df["Seat Type"].str.strip().str.lower() == seat_type]
    filtered_df = filtered_df[filtered_df["Normalized Route (City)"].apply(lambda r: match_route(normalized_input_route, r))]
    return filtered_df

def merge_filtered_results(iata_results, city_results):
    """
    IATA 코드 기반과 도시명 기반 필터링 결과를 병합하고 중복 제거.
    """
    merged_df = pd.concat([iata_results, city_results]).drop_duplicates().reset_index(drop=True)

    # 병합된 데이터를 보기 좋게 정리
    merged_df["Mapped Routes"] = merged_df.apply(
        lambda row: f"{row['Normalized Route (IATA)']} ({row['Normalized Route (City)']})", axis=1
    )

    # 중복된 열 제거
    merged_df = merged_df.drop(columns=["Normalized Route (IATA)", "Normalized Route (City)"])
    return merged_df

# IATA 코드 기반 필터링
filtered_by_iata = filter_data_by_iata(df, user_input)

# 도시명 기반 필터링
filtered_by_city = filter_data_by_city(df, user_input, airport_city_mapping)

# 결과 병합 및 중복 제거
merged_results = merge_filtered_results(filtered_by_iata, filtered_by_city)

if merged_results.empty:
    print("조건에 맞는 데이터가 없습니다. 입력 조건을 확인하세요.")
else:
    print("필터링된 데이터:")
    print(merged_results[["Airline Name", "Mapped Routes"]].head(100))


필터링된 데이터:
                      Airline Name                 Mapped Routes
0              Azerbaijan Airlines              DXB to LHR (nan)
1                         Gulf Air              DXB to LHR (nan)
2                         Gulf Air              DXB to LHR (nan)
3            Royal Brunei Airlines              DXB to LHR (nan)
4   Ukraine International Airlines              LHR to DXB (nan)
5              Azerbaijan Airlines  DXB to LHR (Dubai to London)
6                         Gulf Air  DXB to LHR (Dubai to London)
7                         Gulf Air  DXB to LHR (Dubai to London)
8                 Pegasus Airlines  STN to DXB (London to Dubai)
9                 Pegasus Airlines  STN to DXB (London to Dubai)
10                Pegasus Airlines  STN to DXB (London to Dubai)
11           Royal Brunei Airlines  DXB to LHR (Dubai to London)
12  Ukraine International Airlines  LHR to DXB (London to Dubai)


# 5. 리뷰 감성 분석
필터링된 데이터를 대상으로 BERT 모델을 사용하여 리뷰 감성 점수를 분석합니다.  
이 점수는 이후 추천 점수 계산에 반영됩니다.


In [14]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

def analyze_review_sentiment_pytorch(df, analyze_reviews):
    """
    필터링된 데이터를 대상으로 리뷰 감성 분석 수행.
    """
    if not analyze_reviews:
        return df

    # 디바이스 설정 (GPU 사용 가능 시 GPU로 설정)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # BERT 모델 로드
    model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    model.to(device)

    def predict_sentiment(review):
        """
        단일 리뷰 텍스트에 대해 감성 점수를 예측.
        """
        if pd.isnull(review) or not review.strip():
            return 0  # 빈 리뷰는 0점 처리
        inputs = tokenizer(review, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        return torch.argmax(probs, dim=-1).item() + 1  # 감성 점수: 1~5

    # 진행 상황 출력용
    total_reviews = len(df)
    print(f"총 리뷰 개수: {total_reviews}")

    sentiment_scores = []
    for idx, review in enumerate(df["Review Content"]):
        sentiment_scores.append(predict_sentiment(review))
        progress = (idx + 1) / total_reviews * 100
        print(f"\r진행률: {progress:.2f}% ({idx + 1}/{total_reviews})", end="")

    print("\n리뷰 감성 분석 완료!")
    df["Sentiment Score"] = sentiment_scores
    return df

# 리뷰 감성 분석 실행
filtered_data_with_sentiment = analyze_review_sentiment_pytorch(merged_results, user_input["analyze_reviews"])


총 리뷰 개수: 13
진행률: 100.00% (13/13)
리뷰 감성 분석 완료!


# 6. 추천 점수 계산
사용자의 서비스 선호도와 리뷰 감성 점수를 반영하여 추천 점수를 계산합니다.


In [15]:
def calculate_recommendation_score(df, user_input):
    """
    사용자 선호도 및 감성 점수를 기반으로 추천 점수 계산.
    """
    preferences = user_input["preferences"]
    total_weight = sum(preferences.values())
    weights = {k: v / total_weight for k, v in preferences.items()}  # 가중치 계산

    # 추천 점수 계산
    df["Recommendation Score"] = (
        df["Seat Comfort"] * weights["Seat Comfort"] +
        df["Cabin Staff Service"] * weights["Cabin Staff Service"] +
        df["Food & Beverages"] * weights["Food & Beverages"] +
        df["Inflight Entertainment"] * weights["Inflight Entertainment"] +
        df["Ground Service"] * weights["Ground Service"] +
        df["Wifi & Connectivity"] * weights["Wifi & Connectivity"] +
        df["Value For Money"] * weights["Value For Money"] +
        df["Sentiment Score"]
    )
    return df

# 추천 점수 계산
scored_data = calculate_recommendation_score(filtered_data_with_sentiment, user_input)


# 7. 항공사 추천
계산된 추천 점수를 기반으로 상위 5개의 항공사를 추천합니다.


In [16]:
def recommend_airlines(df):
    """
    추천 점수를 기반으로 상위 항공사를 추천.
    """
    # 항공사별 추천 점수 평균 계산
    airline_scores = df.groupby("Airline Name")["Recommendation Score"].mean().sort_values(ascending=False)
    return airline_scores.head(5)

# 상위 5개 항공사 추천
recommendations = recommend_airlines(scored_data)

print("추천 항공사:")
print(recommendations)

print("\n상세 추천 데이터:")
print(scored_data[["Airline Name", "Mapped Routes", "Recommendation Score"]].head(10))

추천 항공사:
Airline Name
Ukraine International Airlines    7.380952
Azerbaijan Airlines               5.238095
Royal Brunei Airlines             5.142857
Pegasus Airlines                  3.841270
Gulf Air                          2.166667
Name: Recommendation Score, dtype: float64

상세 추천 데이터:
                     Airline Name                 Mapped Routes  \
0             Azerbaijan Airlines              DXB to LHR (nan)   
1                        Gulf Air              DXB to LHR (nan)   
2                        Gulf Air              DXB to LHR (nan)   
3           Royal Brunei Airlines              DXB to LHR (nan)   
4  Ukraine International Airlines              LHR to DXB (nan)   
5             Azerbaijan Airlines  DXB to LHR (Dubai to London)   
6                        Gulf Air  DXB to LHR (Dubai to London)   
7                        Gulf Air  DXB to LHR (Dubai to London)   
8                Pegasus Airlines  STN to DXB (London to Dubai)   
9                Pegasus Airlines  STN 